In [163]:
import os
import re
import random
import string
import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

input_file_path = '/home/kevin/tmp/group1_hic/H3K4me3_R3_TEPIC_08_27_20_15_32_02_600835604_Affinity.txt'
output_dir = '/home/kevin/tmp/group1_hic/bed_files/'

Parse input file as DataFrame

In [164]:
input_df = pd.read_table(input_file_path)
header_df = input_df.iloc[:,0]
value_df = input_df.iloc[:,1:]

In [165]:
position_df = pd.DataFrame()
position_df[['chr', 'start']] = header_df.str.split(':', expand=True)
position_df[['start', 'end']] = position_df.iloc[:,0-1].str.split('-', expand=True)

full_df = pd.concat([position_df, value_df], axis=1)


Get candidates

In [166]:
p = re.compile('STAT', re.IGNORECASE)
tf_candidates = [x for x in full_df.columns if p.match(x)]
print(tf_candidates)
full_df[tf_candidates].median()
full_df

['STAT2_MOUSE.H11MO.0.A:IRF2_MOUSE.H11MO.0.B:IRF1_MOUSE.H11MO.0.A', 'Stat4:Stat3', 'Stat5a::Stat5b:Stat4:Stat3', 'STAT1_MOUSE.H11MO.0.A:IRF3_MOUSE.H11MO.0.A:IRF4_MOUSE.H11MO.0.A:IRF8_MOUSE.H11MO.0.A:STAT2_MOUSE.H11MO.0.A:IRF2_MOUSE.H11MO.0.B:IRF1_MOUSE.H11MO.0.A', 'Stat6:Bcl6:Stat5a::Stat5b:Stat4:Stat3']


,chr,start,end,Rarg:Rarb,Nr2f6(var.2):Rarg:Rarb,Sox11:Sox1,Rxra:Hnf4a,REST_MOUSE.H11MO.0.A:SIN3A,CTCF_MOUSE.H11MO.0.A:CTCFL_MOUSE.H11MO.0.A,Nr2f6:Rxra:Hnf4a,...,Nkx2-5:Nkx2-5(var.2):Nkx3-2:Nkx3-1,SRF_MOUSE.H11MO.0.A:MEF2A_MOUSE.H11MO.0.A:MEF2C_MOUSE.H11MO.0.A:MEF2D_MOUSE.H11MO.0.A,TBP_MOUSE.H11MO.0.A:SRF_MOUSE.H11MO.0.A:MEF2A_MOUSE.H11MO.0.A:MEF2C_MOUSE.H11MO.0.A:MEF2D_MOUSE.H11MO.0.A,HAND1_MOUSE.H11MO.0.C:SMAD2_MOUSE.H11MO.0.A:Smad4:Hand1::Tcf3,DLX5_MOUSE.H11MO.0.C:SP7_MOUSE.H11MO.0.A:Arid3b:Lhx3:Phox2b:PROP1_MOUSE.H11MO.0.C:Barhl1:Pdx1:Lhx8:LHX2_MOUSE.H11MO.0.A:Hoxd3:Dlx1:Prrx2:Lhx4:Dlx3:Dlx4:Nobox:Arx:Alx1:Alx4:Hoxb5:VSX2_MOUSE.H11MO.0.C:Shox2:Msx3:Dlx2:Hoxd8:Hmx1:Hmx3:Hmx2,BARX1_MOUSE.H11MO.0.C:Nkx2-5:Nkx2-5(var.2):Nkx3-2:Nkx3-1,BATF3_MOUSE.H11MO.0.A:IRF7_MOUSE.H11MO.0.C:IRF9_MOUSE.H11MO.0.C:PRDM1_MOUSE.H11MO.0.A:STAT1_MOUSE.H11MO.0.A:IRF3_MOUSE.H11MO.0.A:IRF4_MOUSE.H11MO.0.A:IRF8_MOUSE.H11MO.0.A:STAT2_MOUSE.H11MO.0.A:IRF2_MOUSE.H11MO.0.B:IRF1_MOUSE.H11MO.0.A,Vdr:Nr1h3::Rxra:NR1I2_MOUSE.H11MO.0.C:Rarg(var.2):NR1I3_MOUSE.H11MO.0.C:Nr2e1:NR4A1_MOUSE.H11MO.0.B:RXRB_MOUSE.H11MO.0.C:NR4A2:Pparg::Rxra:PPARA_MOUSE.H11MO.0.A:PPARG_MOUSE.H11MO.0.A:NR2C2_MOUSE.H11MO.0.A:NR1H3_MOUSE.H11MO.0.A:NR2C1_MOUSE.H11MO.0.C:HNF4G_MOUSE.H11MO.0.C:Nr2f6:Rxra:Hnf4a:Nr2f6(var.2):Rarg:Rarb:RARA_MOUSE.H11MO.0.A:RXRG_MOUSE.H11MO.0.B:Rarb(var.2):Esrrg:Esrrb:Nr5a2:Esrra,Pax2:AHR_MOUSE.H11MO.0.B:Ahr::Arnt:HIF1A_MOUSE.H11MO.0.C:EPAS1_MOUSE.H11MO.0.C:ARNT::HIF1A,Rhox11:TBX21_MOUSE.H11MO.0.A:T:TBX3_MOUSE.H11MO.0.B:SMAD3_MOUSE.H11MO.0.B:LYL1_MOUSE.H11MO.0.A:ZEB1_MOUSE.H11MO.0.B:OLIG2_MOUSE.H11MO.0.A:SNAI1_MOUSE.H11MO.0.C:PTF1A_MOUSE.H11MO.0.A:MYF6_MOUSE.H11MO.0.C:Ascl2:Tcf3:Tcf21:ASCL1_MOUSE.H11MO.0.A:SNAI2_MOUSE.H11MO.0.A:Myod1:Tcf12:Myog
0,1,10003185,10003787,5.551080e-18,1.080439e-17,1.571565e-14,9.004629e-13,1.111263e-12,3.667193e-09,1.232622e-12,...,0.035631,0.432756,0.361706,0.140793,4.599214,0.063753,0.208979,0.000611,0.027372,0.341525
1,1,10003946,10004305,4.040050e-17,3.391001e-17,7.154319e-13,1.687278e-13,1.505751e-10,1.912064e-08,2.023710e-13,...,0.001476,0.013081,0.052021,0.886253,5.130918,0.002798,0.126018,0.000098,0.023593,0.104607
2,1,10008807,10009195,2.013556e-15,2.064436e-15,6.547514e-13,3.317250e-09,4.346479e-13,1.110178e-06,3.724953e-09,...,0.062355,0.001708,0.008646,1.161372,1.681913,0.104272,0.022510,0.011183,0.092461,0.112446
3,1,10036598,10039670,2.830072e-13,3.907405e-13,1.554720e-09,6.478507e-11,2.825763e-10,8.689983e-05,7.480777e-11,...,0.270410,0.524621,1.254211,2.334575,33.214793,0.409306,0.478771,0.242861,0.193172,5.958660
4,1,10039734,10040637,2.595467e-16,7.974885e-15,5.789725e-12,4.044499e-11,6.600348e-08,1.957263e-05,5.405392e-11,...,0.005053,0.001140,0.027669,1.173398,3.201364,0.009581,0.025943,0.240126,0.105303,4.188511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34448,Y,90744290,90744601,4.068771e-19,4.182204e-18,4.111573e-16,6.107210e-11,1.558568e-12,4.818779e-04,7.170907e-11,...,0.000257,0.000016,0.000210,0.440417,0.040902,0.000490,0.365060,0.133532,0.011333,0.006710
34449,Y,90761699,90761911,2.784268e-21,6.869608e-21,3.559093e-16,2.008255e-13,2.084659e-14,3.586164e-06,2.648187e-13,...,0.001312,0.002146,0.015401,0.005706,1.311544,0.002330,0.066208,0.002778,0.418821,0.003448
34450,Y,90797829,90798020,3.215739e-21,3.449624e-21,2.516549e-16,5.120382e-11,9.834246e-15,1.913970e-10,5.981921e-11,...,0.007776,0.012858,0.026708,0.027481,3.459966,0.013886,0.037703,0.000394,0.001730,0.000449
34451,Y,90804817,90805131,8.654644e-15,8.080138e-15,2.221874e-08,4.167499e-12,1.236011e-14,1.376066e-08,4.558515e-12,...,0.005069,0.000110,0.002130,0.746285,4.139108,0.008869,0.008944,0.000601,0.001095,0.824339


Convert to bed files

In [167]:
def format_filename(s):
    """Take a string and return a valid filename constructed from the string.
Uses a whitelist approach: any characters not present in valid_chars are
removed. Also spaces are replaced with underscores.

Note: this method may produce invalid filenames such as ``, `.` or `..`
When I use this method I prepend a date string like '2009_01_15_19_46_32_'
and append a file extension like '.txt', so I avoid the potential of using
an invalid filename.

Source: https://gist.github.com/seanh/93666

"""
    valid_chars = ":-_.() %s%s" % (string.ascii_letters, string.digits)
    filename = ''.join(c for c in s if c in valid_chars)
    filename = filename.replace(' ','_') # I don't like spaces in filenames.
    filename = filename.replace(':','-') # I don't like spaces in filenames.
    return filename

for tf in tf_candidates:
    output_file_path = os.path.join(output_dir, format_filename(tf) + '.bed')
    output_df = full_df[['chr', 'start', 'end', tf]]
    output_df[tf] = output_df[tf] * 1000
    output_df.insert(3, 'name', tf)
    output_df.insert(5, 'strand', '.')
    output_df = output_df.loc[output_df[tf] >= 1]
    print(f'TF: {tf} | Min: {output_df[tf].min()} | Max: {output_df[tf].max()} | Mean: {output_df[tf].mean()}')
    print(f'Output file: {output_file_path}')
    output_df.to_csv(output_file_path, index=False, header=False, sep='\t')

TF: STAT2_MOUSE.H11MO.0.A:IRF2_MOUSE.H11MO.0.B:IRF1_MOUSE.H11MO.0.A | Min: 1.0001834609999998 | Max: 3239.2273889999997 | Mean: 78.56995990749321
Output file: /home/kevin/tmp/group1_hic/bed_files/STAT2_MOUSE.H11MO.0.A-IRF2_MOUSE.H11MO.0.B-IRF1_MOUSE.H11MO.0.A.bed
TF: Stat4:Stat3 | Min: 1.000877793 | Max: 1843.6660180000001 | Mean: 103.00479882584256
Output file: /home/kevin/tmp/group1_hic/bed_files/Stat4-Stat3.bed
TF: Stat5a::Stat5b:Stat4:Stat3 | Min: 1.0001872990000003 | Max: 3289.738836 | Mean: 190.8949264777811
Output file: /home/kevin/tmp/group1_hic/bed_files/Stat5a--Stat5b-Stat4-Stat3.bed
TF: STAT1_MOUSE.H11MO.0.A:IRF3_MOUSE.H11MO.0.A:IRF4_MOUSE.H11MO.0.A:IRF8_MOUSE.H11MO.0.A:STAT2_MOUSE.H11MO.0.A:IRF2_MOUSE.H11MO.0.B:IRF1_MOUSE.H11MO.0.A | Min: 1.000268948 | Max: 14708.14352 | Mean: 228.50740566086148
Output file: /home/kevin/tmp/group1_hic/bed_files/STAT1_MOUSE.H11MO.0.A-IRF3_MOUSE.H11MO.0.A-IRF4_MOUSE.H11MO.0.A-IRF8_MOUSE.H11MO.0.A-STAT2_MOUSE.H11MO.0.A-IRF2_MOUSE.H11MO.0.B-IRF

<ipython-input-167-22977be71369>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df[tf] = output_df[tf] * 1000


Create negative peaks

In [168]:
# Sort DF
sorted_full_df = full_df
sorted_full_df[['start', 'end']] = sorted_full_df[['start', 'end']].astype(int)
sorted_full_df = sorted_full_df.sort_values('start')

In [169]:
# Calculate mean of peak width
p_peaks = sorted_full_df['end'] - sorted_full_df['start']
peak_mean = p_peaks.mean()
peak_count = sorted_full_df.shape[1]
print(f'Positive peak mean: {peak_mean}')
print(f'Positive peak min: {p_peaks.min()}')
print(f'Positive peak max: {p_peaks.max()}')

# Calculate negative space
n_peaks = pd.DataFrame({'chr': sorted_full_df['chr'], 'start': sorted_full_df['end'].shift(1), 'end': sorted_full_df['start']}).dropna()
n_output_df = pd.DataFrame(columns=['chr','start','end','name','score','strand'])

np.random.seed(1)
random.seed(1)

peak_length_random = np.random.normal(loc = peak_mean, scale = 500, size = int(len(p_peaks)*1000)).astype(int)
peak_length_random = peak_length_random[peak_length_random > 10]
print(f'RANDOM {peak_length_random}')
print(f'RANDOM min {peak_length_random.min()}')
print(f'RANDOM max {peak_length_random.max()}')
counter = 0
for index, row in tqdm.tqdm(n_peaks.iterrows(), total=n_peaks.shape[0]):
    gap_length = row['end'] - row['start']
    peak_length = 0

    # Get peak length smaller than gap
    for _ in range(1000):
        peak_length = peak_length_random[counter]
        counter += 1
        if peak_length <= gap_length:
            break

    # Get start offset
    offset = 0
    try:
        offset = random.randint(0, int(gap_length - peak_length))
    except ValueError:
        continue

    # Add row to df
    row_start = row['start'] + offset

    n_output_df.loc[index] = [row['chr'], row_start, row_start + peak_length, 'n_peak', 1, '.']

# No digits
n_output_df[["start", "end"]] = n_output_df[["start", "end"]].astype(int)
# Sort
n_output_df = n_output_df.sort_values(['chr', 'start'])

print(f"MEAN {(n_output_df['end'] - n_output_df['start']).mean()}")
print(f"Length {len(n_output_df['start'])}")
n_output_df.to_csv(os.path.join(output_dir, 'negative_peak.bed'), index=False, header=False, sep='\t')

Positive peak mean: 919.2600063855107
Positive peak min: 191
Positive peak max: 12084
RANDOM [1731  613  655 ... 1511 1236  775]
RANDOM min 11
RANDOM max 3775
MEAN 596.7403741496598
Length 29400


100%|██████████| 34452/34452 [04:54<00:00, 116.86it/s]
